# sport

## init code

In [107]:
import io
from glob import glob
import pandas as pd
from pathlib import Path
from pandas_plots import tbl

dir_data_in=Path("../data/in")
dir_data_out=Path("../data/out")

file_csv_list=glob(f"{dir_data_in}/*.csv")
assert len(file_csv_list) == 1, f"❌ Expected 1 csv file, found {len(file_csv_list)}"

file_csv = file_csv_list[0]
file_xls="Register_fertig.xlsx"

## load data

In [80]:
# * read excel
df_xls=pd.read_excel(dir_data_in/file_xls, sheet_name="Tabelle fertig")

# * try reading csv file
with open(dir_data_in/file_csv, encoding="windows-1252") as f:
    csv=f.read()

In [81]:
# Replace commas inside double quotes with semicolons
def replace_delimiter_outside_quotes(
    input: str, delimiter_old: str = ",", delimiter_new: str = ";", quotechar: str = '"'
):
    outside_quotes = True
    output = ""
    for char in input:
        if char == quotechar:
            outside_quotes = not outside_quotes
        elif outside_quotes and char == delimiter_old:
            char = delimiter_new
        output += char
    return output

replace_delimiter_outside_quotes(
    'Participant ID,"Survey Identifier","Survey Timestamp",Datum,CTCAE-Bewertung,"Was war das AE? (Schmerz, Schürfwunde etc.)"'
)

'Participant ID;"Survey Identifier";"Survey Timestamp";Datum;CTCAE-Bewertung;"Was war das AE? (Schmerz, Schürfwunde etc.)"'

In [82]:
# Replace commas inside double quotes with semicolons
def reduce_quotes_in_string(input: str):
    output = ""
    len_=len(input)
    i = 0
    while i < len_:
        # * 
        if input[i] == '"' and i+1 < len_ and input[i+1] != '"':
            output += ''
            i = i+1
        elif input[i] == '"'and i+1 < len_ and input[i+1] == '"':
            output+= '"'
            i = i+2
        else:
            output += input[i]
            i=i+1

    return output

lol=reduce_quotes_in_string('4,,""2022-11-08 09:19:23"",,1,""Schmerzen im Hüft- und Kniebereich (kurzzeitig)"",,,No,,Ja,Checked,Unchecked,Unchecked,Unchecked,mild,""weil der Junge nicht geweint hat"",No,No,,No,No,No,,No,,,No,,,,,,,,Checked,Unchecked,Checked,Unchecked,Unchecked,,No,No,,""max. 3 Stunden"",""Luftballonspiel im Stehen"	 Kind hat sich gestreckt und dabei das Gleichgewicht verloren	" Fall auf Seite"",No,Yes,""unter 5 Minuten"",Yes,""Kind hat Anweisung bekommen sich nicht zu strecken und der Ballon wurde vom Therapeuten tiefer gespielt"",Yes,Unchecked,Checked,Unchecked,Yes,Yes,Yes,Akuttherapie,Einzel,""6 bis 9 Jahre"",No,No,Klinikflur,,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,""1. Hälfe"",Checked,Unchecked,Unchecked,Unchecked,Unchecked,,Essen,,Complete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Incomplete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,Incomplete')
lol2=reduce_quotes_in_string('5,,""2022-11-08 09:31:05"",,2,""Schwindel und Fieber"",,Nein,No,,Nein,Unchecked,Unchecked,Unchecked,Unchecked,,,No,Yes,,No,No,No,,Yes,""Gabe von fiebersenkenden Medikamenten."",,No,,,,Nein,,Ja,,Checked,Unchecked,Unchecked,Unchecked,Unchecked,,No,No,,""max. 3 Stunden"",""Beim leichter Mobilisation und Aktivierung hat das Kind Schwindel rückgemeldet"	" es wurde Fieber gemessen und Fieber festgestellt"",No,No,,Yes,,No,Checked,Unchecked,Unchecked,No,Yes,No,Akuttherapie,Einzel,""10 bis 14 Jahre"",No,No,Pat.-Zimmer,,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,""2. Hälfte"",Unchecked,Unchecked,Checked,Unchecked,Checked,""red. AZ"",Essen,,Complete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Incomplete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,Incomplete')
lol3=reduce_quotes_in_string('43,,,2023_10_10,,""Schmerzen ain der Glutealmuskulatur und im Steiß"",,""Weiß nicht"",No,,Ja,Unchecked,Unchecked,Unchecked,Unchecked,,,No,No,,No,No,No,,No,,,No,,,,Nein,,Nein,,Unchecked,Unchecked,Unchecked,Unchecked,Checked,,No,,,,""Betroffener wollte sich aufrichten, um Spazieren zu gehen"	 bevor er sich aufgerichtet hatte, kam es zu einer Schmerz-Attacke	 Arzt dazugerufen	 dieser hat ihn untersucht und Bewegungen in verschiedene Richtungen untersucht	 alles war uauffällig	 Pat. hatte das seit der letzten LP bereits 2 mal	" Bruder von ihn, hatte ihn zuvor am Rücken massiert"",Yes,,,,,Yes,Checked,Checked,Unchecked,No,Yes,Yes,Akuttherapie,Einzel,""15 bis 18 Jahre"",No,No,Pat.-Zimmer,,Unchecked,Unchecked,Checked,Checked,Unchecked,Unchecked,""1. Hälfe"",Unchecked,Unchecked,Checked,Unchecked,Checked,""nach KMT"	 hatte stark abgenommen	" schlechter AZ!"",Essen,,Incomplete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Incomplete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,,,,,,,,,,,,,,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,,,Incomplete')
lol3

'43,,,2023_10_10,,"Schmerzen ain der Glutealmuskulatur und im Steiß",,"Weiß nicht",No,,Ja,Unchecked,Unchecked,Unchecked,Unchecked,,,No,No,,No,No,No,,No,,,No,,,,Nein,,Nein,,Unchecked,Unchecked,Unchecked,Unchecked,Checked,,No,,,,"Betroffener wollte sich aufrichten, um Spazieren zu gehen\t bevor er sich aufgerichtet hatte, kam es zu einer Schmerz-Attacke\t Arzt dazugerufen\t dieser hat ihn untersucht und Bewegungen in verschiedene Richtungen untersucht\t alles war uauffällig\t Pat. hatte das seit der letzten LP bereits 2 mal\t Bruder von ihn, hatte ihn zuvor am Rücken massiert",Yes,,,,,Yes,Checked,Checked,Unchecked,No,Yes,Yes,Akuttherapie,Einzel,"15 bis 18 Jahre",No,No,Pat.-Zimmer,,Unchecked,Unchecked,Checked,Checked,Unchecked,Unchecked,"1. Hälfe",Unchecked,Unchecked,Checked,Unchecked,Checked,"nach KMT\t hatte stark abgenommen\t schlechter AZ!",Essen,,Incomplete,,,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchec

In [83]:
# csv = csv.replace("\t", "")
l = csv.splitlines()
l2 = []
for line in l[:]:
    if line[:1] == '"':
        line = line[1:-1]
    line=reduce_quotes_in_string(line)
    line=replace_delimiter_outside_quotes(line)
    line=line.replace('"', '')
    l2.append(line)
    # print(f'{line.count(";")} {line}')
s = "\n".join(l2).replace('\t', '')

df_csv = pd.read_csv(
    io.StringIO(s),
    sep=";",
    encoding="windows-1252",
)


In [84]:
# tbl.describe_df(df_csv.iloc[:,:], "df_csv", top_n_uniques=5, top_n_chars_in_index=5, use_plot=False)

## reduce

In [87]:

cols_no_value = [col for col in df_csv.columns if df_csv[col].isna().all()]
display(df_csv[cols_no_value][:3])

# _few_values_threshold = 3
# cols_few_values = [col for col in df_csv.columns if (df_csv[col].notna().sum() < _few_values_threshold and df_csv[col].notna().any())]
# df_csv[cols_few_values][:10]


,Survey Identifier,Wie lange dauerte(n) die Konsequenz(en) an?,Wie lange dauerte(n) die Konsequenz(en) an?.2,Was genau?,Wie lange dauerte(n) die Konsequenz(en) an?.5,Survey Timestamp.1,An welchem NAOK-Standort ist das AE aufgetreten?.1,Erläuterung Sonstiges.1,Wie lange dauerte(n) die Konsequenz(en) an?.7,Das AE ist durch den Sport neu aufgetreten..1,...,Wie alt war der/die Betroffene zum Zeitpunkt des AE?.2,Fand das Sportprogramm-Online statt?.1,Fand das AE im Rahmen einer Sporttestung statt?.1,In welchem Setting ist das AE aufgetreten?.2,Erläuterung Sonstiges.5,Was ist genau passiert?.2,Erläuterung Sonstiges.6,In welcher Phase der geplanten Sporteinheit trat das AE auf?.2,"Glaubst Du, dass das AE mit dem Sport assoziiert ist?",Anmerkung.2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
df_csv = df_csv[list(set(df_csv.columns) - set(cols_no_value))]

In [100]:
cols_excel=df_xls.columns
cols_csv_and_excel = list(set(cols_excel) & set(df_csv.columns))

df_csv = df_csv[cols_csv_and_excel]
df_csv['CTCAE-Bewertung'] = df_csv['CTCAE-Bewertung'].astype('Int64')

In [103]:
df_csv

,Wie wurde die BT angepasst?,In welchem Maße?,Hat jemand mit Expertise die Situation geprüft?,Datum,Wurde das Kind getröstet?,Das AE ist durch den Sport neu aufgetreten.,"Glauben Sie, dass das AE mit dem Sport assoziiert ist?",Fand das Sportprogramm-Online statt?,Wie stark waren diese Schmerzen?,Das AE war vor dem Sport bereits vorhanden und hat sich verstärkt.,...,Kam es zu einer deutlichen Verzögerung bei der Einhaltung des medizinischen Therapieprotokolls?,Kam es zum Tod?,Wurde das Kind nach dem AE observiert?,Wurde die Bewegungseinheit angepasst?,Wie kommen Sie zu dieser Einschätzung?,"Was war das AE? (Schmerz, Schürfwunde etc.)",Wurde die PECH-Regel angewendet?,Waren die Maßnahmen ausschließlich pädagischer/pysachologischer Natur? (choice=ja),Complete?,An welchem NAOK-Standort ist das AE aufgetreten?
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unchecked,Incomplete,NaN
1,NaN,NaN,NaN,NaN,No,Yes,Yes,No,NaN,No,...,No,No,Nein,No,NaN,Schürfwunde an der Wange,Nein,Unchecked,Complete,Essen
2,NaN,NaN,NaN,NaN,Yes,No,No,No,stark,No,...,No,No,Nein,NaN,Einschätzung von außen durch Sporttherapeutin,strake Übelkeit,Nein,Checked,Complete,Essen
3,Kind hat Anweisung bekommen sich nicht zu stre...,NaN,NaN,NaN,Yes,Yes,Yes,No,mild,Yes,...,No,No,NaN,Yes,weil der Junge nicht geweint hat,Schmerzen im Hüft- und Kniebereich (kurzzeitig),NaN,Unchecked,Complete,Essen
4,NaN,NaN,NaN,NaN,No,No,No,No,NaN,Yes,...,No,No,Ja,Yes,NaN,Schwindel und Fieber,Nein,Checked,Complete,Essen
5,NaN,NaN,NaN,NaN,No,Yes,No,No,NaN,No,...,No,No,Nein,No,NaN,Erbrechen,Nein,Checked,Complete,Essen
6,Bis zum Ende der Reha: Sport nur noch im Sitzen,mittelmäßig,NaN,NaN,Yes,Yes,Yes,No,mittel,No,...,No,No,Ja,NaN,"Schmerzen waren da, aber gehen war möglich hat...",Bruch re. oberere Srunggelenk Tibiakopf,Nein,Checked,Complete,Katharienenhöhe
7,langfristig: Intensität noch mehr reduziert,stark,NaN,NaN,No,No,No,No,sehr stark,Yes,...,No,No,Nein,Yes,Äußerung der Pat.,Schmerzen und Verspannungen,Nein,Checked,Complete,Essen
8,NaN,NaN,NaN,NaN,Yes,No,No,No,NaN,Yes,...,No,No,Nein,NaN,NaN,plötzliche Übelkeit und Erbrechen während der ...,Nein,Checked,Complete,München
9,NaN,moderat,NaN,NaN,Yes,No,No,No,mittel,Yes,...,No,No,Weiß nicht,NaN,Kind hat geweint,steckende Rückenschmerzen nach Abrutschen von ...,Nein,Checked,Complete,München


## 💾 data

In [109]:
df_csv.reset_index().to_csv(Path(dir_data_out/"df_csv.csv"), index=False, encoding="utf-8-sig")